# Day 4 - Working With Files and Calling APIs

Welcome to Day 4 of our Python course. Today we will:

- Get comfortable with **files and directories** on your computer
- Use common file methods: `open`, `read`, `readline`, `readlines`, `write`, `writelines`, `close`
- Learn about **context managers** (`with open(...)`) for safe file handling
- Explore the `os` module for:
  - Listing directories with `os.listdir()`
  - Walking directory trees with `os.walk()`
  - Creating directories with `os.makedirs()`
  - Deleting and renaming files with `os.remove()` and `os.rename()`
  - Checking file and directory existence with `os.path`
  - Getting file metadata with `os.stat()`
- Use **object-oriented file handling** with `pathlib.Path`
- Move and copy files with `shutil`
- Work with **temporary files** using `tempfile`
- Get a high-level overview of the **HTTP protocol**
- Use the `requests` library for simple HTTP GET and POST calls
- See how to register for **OpenRouter** and call an LLM API from Python

## Daily agenda and course flow

**09:00 - 10:30 (1h 30m)**
- Why work with files and APIs
- Basic file handling and common file methods
- Safe file handling with context managers

**10:30 - 10:45 (15m)**  
- Short break

**10:45 - 12:00 (1h 15m)**
- Directory handling with `os` and `os.path`
- Walking directories with `os.walk`
- Creating, renaming, deleting files and folders (`os`, `os.makedirs`)
- File metadata with `os.stat`

**12:00 - 13:00 (1h)**  
- Lunch break

**13:00 - 14:45 (1h 45m)**
- `pathlib` for object-oriented paths
- Moving and copying with `shutil`
- Temporary files with `tempfile`
- Short combined exercises

**14:45 - 15:00 (15m)**  
- Short break

**15:00 - 16:30 (1h 30m)**
- HTTP protocol overview
- `requests` basics
- OpenRouter registration and simple LLM call
- Complex combined example using file operations and HTTP
- Day summary and Q&A

### Helpful references

- File objects: https://docs.python.org/3/tutorial/inputoutput.html#reading-and-writing-files
- `os` module: https://docs.python.org/3/library/os.html
- `os.path`: https://docs.python.org/3/library/os.path.html
- `pathlib`: https://docs.python.org/3/library/pathlib.html
- `shutil`: https://docs.python.org/3/library/shutil.html
- `tempfile`: https://docs.python.org/3/library/tempfile.html
- HTTP basics (MDN): https://developer.mozilla.org/en-US/docs/Web/HTTP/Overview
- `requests` library docs: https://requests.readthedocs.io/en/latest/
- OpenRouter: https://openrouter.ai/


## 1. Why work with files and APIs?

In almost every real-world Python project you will:

- **Read input** from files (CSV exports, logs, config files, reports)
- **Write output** to files (results, logs, generated documents)
- Talk to other systems over the network using **HTTP APIs** (web services, LLMs, payment providers, etc.)

You can think of files and APIs as two major ways your program connects to the outside world:

- Files: slower but often large, persistent data living on disk
- HTTP APIs: remote services that can answer questions, store data, or perform actions for you

Today we learn the basic tools for both.

### Trivia

- Under the hood, Python file objects wrap OS level file descriptors. Most operations eventually become system calls like `open`, `read`, `write` in the operating system.
- HTTP is a **text-based protocol**. Even when you send JSON, underneath it is just text sent over TCP.


## 2. Basic file handling and common methods

Python uses the built-in function `open()` to work with files.

```python
f = open("example.txt", "w", encoding="utf-8")
```

Important file modes:

- `'r'` - read (file must exist)
- `'w'` - write (overwrite or create)
- `'a'` - append (add to end, create if missing)
- `'rb'`, `'wb'` - binary read/write (images, non-text data)

Common file methods:

- `f.read()` - read the whole file as a single string
- `f.readline()` - read one line at a time
- `f.readlines()` - read all lines into a list of strings
- `f.write(text)` - write a string to the file (returns number of bytes/characters)
- `f.writelines(list_of_strings)` - write a list of strings
- `f.close()` - release the file resource

Using these correctly is essential to avoid data loss and file corruption.

Documentation: https://docs.python.org/3/tutorial/inputoutput.html#methods-of-file-objects


In [ ]:
# Example: writing and reading a simple text file with common methods

filename = "day4_example_basic.txt"

# 1. Write some lines to the file
f = open(filename, "w", encoding="utf-8")
f.write("First line\n")
f.writelines(["Second line\n", "Third line\n"])
# Always close when done
f.close()

# 2. Read the whole file at once
f = open(filename, "r", encoding="utf-8")
contents = f.read()
print("--- read() output ---")
print(contents)
f.close()

# 3. Read line by line
f = open(filename, "r", encoding="utf-8")
print("--- readline() calls ---")
print(repr(f.readline()))
print(repr(f.readline()))
print(repr(f.readline()))
f.close()

# 4. Read all lines into a list
f = open(filename, "r", encoding="utf-8")
lines = f.readlines()
print("--- readlines() output ---")
print(lines)
f.close()


### ✏ Exercise (easy): Save and load a short note

Create a small script that:

1. Asks the user for a short note using `input()`.
2. Opens a file called `my_note.txt` in write mode and writes the note into it (do not forget the newline).
3. Closes the file.
4. Opens `my_note.txt` again in read mode.
5. Reads the content with `read()` and prints it.

Use the methods shown above: `open`, `write`, `read`, `close`.


In [ ]:
# TODO: implement saving and loading a short note.

# note = input("Enter a short note: ")

# 1. Open my_note.txt for writing and save the note
# f = ...
# ...
# f.close()

# 2. Open my_note.txt for reading and print its contents
# f = ...
# ...
# f.close()


In [ ]:
# Reference solution: saving and loading a short note

note = "This is a test note written on day 4."  # replace with input(...) for interactive use

# 1. Save the note
f = open("my_note.txt", "w", encoding="utf-8")
f.write(note + "\n")
f.close()

# 2. Load and print the note
f = open("my_note.txt", "r", encoding="utf-8")
loaded = f.read()
f.close()

print("Loaded note:")
print(loaded)


## 3. Context managers recap: with open(...)

From Day 3 you already know context managers and the `with` statement. With files, this is the preferred pattern:

```python
with open("example.txt", "r", encoding="utf-8") as f:
    data = f.read()
    # use data
```

Advantages:

- The file is closed automatically when the `with` block ends.
- It works even if an exception is raised inside the block.

Internally, the file object implements special methods `__enter__` and `__exit__`, which make it a context manager.

Documentation: https://docs.python.org/3/reference/datamodel.html#context-managers


In [ ]:
# Example: using with open for safe reading

filename = "day4_with_example.txt"

# Write something quickly
with open(filename, "w", encoding="utf-8") as f:
    f.write("Line 1\n")
    f.write("Line 2\n")

# Now read safely
with open(filename, "r", encoding="utf-8") as f:
    print("Contents of day4_with_example.txt:")
    for line in f:
        print(repr(line))


---
# Short break (10:30-10:45)

---

## 4. Working with directories using os and os.listdir()

The `os` module provides functions for interacting with the operating system.

`os.listdir(path)` returns a list of entries in the given directory.

```python
import os
entries = os.listdir(".")  # current directory
for name in entries:
    print(name)
```

You can combine this with `os.path` to check which entries are files or directories.

Documentation:

- `os` module: https://docs.python.org/3/library/os.html
- `os.listdir`: https://docs.python.org/3/library/os.html#os.listdir


In [ ]:
# Example: list current directory contents

import os

print("Entries in current directory:")
for name in os.listdir("."):
    print(" -", name)


### ✏ Exercise (easy): List all .txt files in the current directory

Write a script that:

1. Uses `os.listdir(".")` to get all entries in the current directory.
2. Filters them to only keep entries that end with `.txt`.
3. Prints the `.txt` filenames, one per line.

You can use basic string methods like `name.endswith(".txt")`.


In [ ]:
# TODO: list all .txt files in the current directory.

#import os

# for name in os.listdir("."):
#     # if name ends with .txt, print it
#     ...


In [ ]:
# Reference solution: list all .txt files

import os

print(".txt files in current directory:")
for name in os.listdir("."):
    if name.endswith(".txt"):
        print(name)


## 5. Walking directory trees with os.walk()

`os.walk(top)` lets you traverse a directory tree.

It yields a 3-tuple `(dirpath, dirnames, filenames)` for each directory.

```python
import os
for dirpath, dirnames, filenames in os.walk("."):
    print("Directory:", dirpath)
    print("Subdirectories:", dirnames)
    print("Files:", filenames)
```

This is very useful for tasks like:

- Finding all Python files under a project
- Searching for specific filenames
- Computing statistics over a whole directory tree

Documentation: https://docs.python.org/3/library/os.html#os.walk


In [ ]:
# Example: count all .py files under the current directory

import os

py_count = 0
for dirpath, dirnames, filenames in os.walk("."):
    for filename in filenames:
        if filename.endswith(".py"):
            py_count += 1

print("Number of .py files under current directory:", py_count)


### ⚡ Exercise (advanced): Report file counts per directory

Using `os.walk(".")`, write a script that:

1. Iterates over all directories starting from the current directory.
2. For each directory, counts how many files it has (just `len(filenames)`).
3. Prints lines like:
   - `"./: 5 files"`
   - `"./subdir: 3 files"`

Use only `os.walk` and the basics shown above.


In [ ]:
# TODO: print how many files each directory contains.

#import os

# for dirpath, dirnames, filenames in os.walk("."):
#     # compute number of files and print dirpath and count
#     ...


In [ ]:
# Reference solution: file counts per directory

import os

for dirpath, dirnames, filenames in os.walk("."):
    count = len(filenames)
    print(f"{dirpath}: {count} files")


## 6. Checking file and directory existence with os.path

`os.path` contains helpers for working with paths.

Useful functions:

- `os.path.exists(path)` - does this path exist at all?
- `os.path.isfile(path)` - is it an existing regular file?
- `os.path.isdir(path)` - is it an existing directory?
- `os.path.join(a, b)` - join path components in an OS independent way

```python
import os
path = "my_note.txt"
if os.path.exists(path):
    print("Path exists")
```

Documentation: https://docs.python.org/3/library/os.path.html


In [ ]:
# Example: checking path types

import os

paths_to_check = ["my_note.txt", ".", "definitely_not_existing_123.txt"]

for path in paths_to_check:
    print(f"Checking {path!r}:")
    print("  exists:", os.path.exists(path))
    print("  is file:", os.path.isfile(path))
    print("  is dir:", os.path.isdir(path))


### ✏ Exercise (easy): Safely read a file if it exists

Write a script that:

1. Asks the user for a filename.
2. Uses `os.path.exists` and `os.path.isfile` to check if it is an existing file.
3. If it is a file, opens it using `with open(..., "r", encoding="utf-8")` and prints the first line.
4. If not, prints a friendly message.

Use the patterns shown above and the context manager pattern from Day 3.


In [ ]:
# TODO: safely read a file if it exists.

#import os

# filename = input("Enter filename to read: ")

# if ...:  # check if file exists and is a file
#     with open(filename, "r", encoding="utf-8") as f:
#         # read and print first line
#         ...
# else:
#     print("File does not exist or is not a regular file.")


In [ ]:
# Reference solution: safely read a file if it exists

import os

filename = "my_note.txt"  # replace with input(...) for interactive use

if os.path.exists(filename) and os.path.isfile(filename):
    with open(filename, "r", encoding="utf-8") as f:
        first_line = f.readline()
    print("First line:", first_line)
else:
    print("File does not exist or is not a regular file.")


## 7. Deleting files with os.remove()

If you want to delete a file, you can use `os.remove(path)`.

```python
import os
os.remove("old_file.txt")
```

Be careful: this operation is permanent from Python's point of view. There is no built-in undo.

Documentation: https://docs.python.org/3/library/os.html#os.remove


In [ ]:
# Example: create and then delete a file

import os

filename = "day4_delete_me.txt"

# Create the file
with open(filename, "w", encoding="utf-8") as f:
    f.write("To be deleted.\n")

print("Created", filename, "exists:", os.path.exists(filename))

# Delete it
os.remove(filename)
print("After deletion, exists:", os.path.exists(filename))


### ✏ Exercise (easy): Ask before deleting

Write a script that:

1. Asks the user for a filename to delete.
2. Checks with `os.path.isfile` if it is an existing file.
3. If yes, asks for confirmation with `input("Are you sure? (y/n): ")`.
4. If the user types `"y"`, delete the file with `os.remove` and print a confirmation.
5. Otherwise, print that nothing was deleted.
6. If the path is not a file, print an error message.


In [ ]:
# TODO: implement safe delete with confirmation.

#import os

# filename = input("Enter filename to delete: ")

# if os.path.isfile(filename):
#     answer = input("Are you sure? (y/n): ")
#     if answer == "y":
#         # delete file
#         ...
#     else:
#         print("Nothing deleted.")
# else:
#     print("The given path is not an existing file.")


In [ ]:
# Reference solution: safe delete with confirmation

import os

filename = "day4_delete_test.txt"  # replace with input(...) for interactive use

# create file for the demo
if not os.path.exists(filename):
    with open(filename, "w", encoding="utf-8") as f:
        f.write("delete test\n")

if os.path.isfile(filename):
    answer = "y"  # replace with input("Are you sure? (y/n): ")
    if answer == "y":
        os.remove(filename)
        print("File deleted.")
    else:
        print("Nothing deleted.")
else:
    print("The given path is not an existing file.")


## 8. Renaming files with os.rename()

You can rename or move a file with `os.rename(src, dst)`.

```python
import os
os.rename("old_name.txt", "new_name.txt")
```

If `dst` includes a different directory, this effectively moves the file.

Documentation: https://docs.python.org/3/library/os.html#os.rename


In [ ]:
# Example: rename a file

import os

old_name = "day4_rename_old.txt"
new_name = "day4_rename_new.txt"

# Create file if not exists
if not os.path.exists(old_name) and not os.path.exists(new_name):
    with open(old_name, "w", encoding="utf-8") as f:
        f.write("rename me\n")

if os.path.exists(old_name):
    os.rename(old_name, new_name)
    print(f"Renamed {old_name!r} to {new_name!r}")
else:
    print("Nothing to rename.")


### ✏ Exercise (easy): Create a backup copy name with .bak

Write a script that:

1. Asks the user for an existing filename.
2. Checks with `os.path.isfile` that it exists.
3. Builds a new name by adding `.bak` to the end (for example `config.txt` -> `config.txt.bak`).
4. Uses `os.rename` to rename the original file to the backup name.
5. Prints the old and new names.


In [ ]:
# TODO: rename a file to add .bak extension.

#import os

# filename = input("Enter filename to backup: ")

# if os.path.isfile(filename):
#     backup_name = filename + ".bak"
#     # rename to backup_name
#     ...
# else:
#     print("File does not exist.")


In [ ]:
# Reference solution: rename a file to add .bak

import os

filename = "day4_backup_test.txt"  # replace with input(...) for interactive use

# create file for demo
if not os.path.exists(filename):
    with open(filename, "w", encoding="utf-8") as f:
        f.write("backup test\n")

if os.path.isfile(filename):
    backup_name = filename + ".bak"
    os.rename(filename, backup_name)
    print(f"Renamed {filename!r} to {backup_name!r}")
else:
    print("File does not exist.")


## 9. Creating nested directories with os.makedirs()

`os.makedirs(path, exist_ok=False)` creates intermediate directories as needed.

```python
import os
os.makedirs("logs/2025/11/17", exist_ok=True)
```

With `exist_ok=True`, it will not raise an error if the directory already exists.

Documentation: https://docs.python.org/3/library/os.html#os.makedirs


In [ ]:
# Example: create a nested directory structure

import os

nested_dir = os.path.join("day4_output", "logs", "2025", "11", "17")
os.makedirs(nested_dir, exist_ok=True)

print("Created or confirmed directory:", nested_dir)
print("Exists:", os.path.isdir(nested_dir))


### 🏃‍♂️ Exercise (medium): Create per-user folders

Write a script that:

1. Has a list of usernames, for example `users = ["anna", "bela", "csaba"]`.
2. For each username, creates a folder `day4_users/<username>/inbox` using `os.makedirs` with `exist_ok=True`.
3. Prints the full path for each created inbox directory.

Use `os.path.join` to build paths.


In [ ]:
# TODO: create per-user inbox directories.

#import os

# users = ["anna", "bela", "csaba"]

# base = "day4_users"

# for user in users:
#     inbox_path = os.path.join(base, user, "inbox")
#     # create the directory tree
#     ...
#     print("Created inbox:", inbox_path)


In [ ]:
# Reference solution: per-user inbox directories

import os

users = ["anna", "bela", "csaba"]
base = "day4_users"

for user in users:
    inbox_path = os.path.join(base, user, "inbox")
    os.makedirs(inbox_path, exist_ok=True)
    print("Created inbox:", inbox_path)


## 10. File metadata with os.stat()

`os.stat(path)` returns an object with various metadata:

- `st_size` - size in bytes
- `st_mtime` - last modification time (seconds since Unix epoch)
- `st_ctime` - creation time on some systems

```python
import os
info = os.stat("my_note.txt")
print(info.st_size)
```

Documentation: https://docs.python.org/3/library/os.html#os.stat


In [ ]:
# Example: show size and modification time

import os
import time

filename = "my_note.txt"

if os.path.exists(filename):
    info = os.stat(filename)
    print("Size in bytes:", info.st_size)
    print("Last modified (raw):", info.st_mtime)
    print("Last modified (readable):", time.ctime(info.st_mtime))
else:
    print("File", filename, "does not exist.")


### ⚡ Exercise (advanced): Find the largest file in a directory

Write a script that:

1. Asks the user for a directory path.
2. Uses `os.listdir` and `os.path.join` to iterate over entries in that directory.
3. For each entry that is a file, uses `os.stat` to get its size.
4. Finds the file with the largest size and prints its name and size.
5. If there are no files in the directory, print a message.


In [ ]:
# TODO: find the largest file in a directory.

#import os

# folder = input("Enter directory path: ")

# if not os.path.isdir(folder):
#     print("Not a directory.")
# else:
#     largest_name = None
#     largest_size = 0
#     for name in os.listdir(folder):
#         full_path = os.path.join(folder, name)
#         if os.path.isfile(full_path):
#             # get size with os.stat
#             ...
#     if largest_name is None:
#         print("No files in this directory.")
#     else:
#         print("Largest file:", largest_name, "with size", largest_size, "bytes")


In [ ]:
# Reference solution: largest file in a directory

import os

folder = "."  # replace with input(...) for interactive use

if not os.path.isdir(folder):
    print("Not a directory.")
else:
    largest_name = None
    largest_size = 0
    for name in os.listdir(folder):
        full_path = os.path.join(folder, name)
        if os.path.isfile(full_path):
            size = os.stat(full_path).st_size
            if largest_name is None or size > largest_size:
                largest_name = name
                largest_size = size
    if largest_name is None:
        print("No files in this directory.")
    else:
        print("Largest file:", largest_name, "with size", largest_size, "bytes")


---
# Lunch break (12:00-13:00)

---

## 11. Object-oriented paths with pathlib

The `pathlib` module provides an object-oriented way to handle paths.

Key concepts:

- `Path` objects represent file system paths.
- You can use `/` to join paths in an OS independent way.
- Methods like `.exists()`, `.is_file()`, `.is_dir()`, `.glob()` help you work with files.

```python
from pathlib import Path
base = Path(".")
for path in base.glob("*.txt"):
    print(path, "exists?", path.exists())
```

Documentation: https://docs.python.org/3/library/pathlib.html


In [ ]:
# Example: list .txt files using pathlib

from pathlib import Path

base = Path(".")

print(".txt files with pathlib:")
for path in base.glob("*.txt"):
    print(" -", path, "file?", path.is_file())


### 🧪 Exercise (medium): Separate files and directories

Using `pathlib.Path`, write a script that:

1. Creates a `Path` object for the current directory.
2. Iterates over all entries using `.iterdir()`.
3. Collects two lists: one for files, one for directories.
4. Prints both lists.

Use the methods `.is_file()` and `.is_dir()`.


In [ ]:
# TODO: separate files and directories using pathlib.

#from pathlib import Path

# base = Path(".")
# files = []
# dirs = []

# for path in base.iterdir():
#     if path.is_file():
#         # add to files
#         ...
#     elif path.is_dir():
#         # add to dirs
#         ...

# print("Files:", files)
# print("Directories:", dirs)


In [ ]:
# Reference solution: separate files and directories with pathlib

from pathlib import Path

base = Path(".")
files = []
dirs = []

for path in base.iterdir():
    if path.is_file():
        files.append(path.name)
    elif path.is_dir():
        dirs.append(path.name)

print("Files:", files)
print("Directories:", dirs)


## 12. Moving and copying files with shutil

The `shutil` module provides high-level file operations.

Common functions:

- `shutil.copy(src, dst)` - copy file contents and permissions
- `shutil.move(src, dst)` - move a file or directory

```python
import shutil
shutil.copy("source.txt", "copy.txt")
shutil.move("copy.txt", "backup/copy.txt")
```

Documentation: https://docs.python.org/3/library/shutil.html


In [ ]:
# Example: copy and move a file

import os
import shutil

os.makedirs("day4_shutil", exist_ok=True)

src = "day4_shutil_source.txt"
copy_target = os.path.join("day4_shutil", "copy.txt")
move_target = os.path.join("day4_shutil", "moved_copy.txt")

# create source file
with open(src, "w", encoding="utf-8") as f:
    f.write("content for shutil demo\n")

# copy to folder
shutil.copy(src, copy_target)
print("Copied to", copy_target)

# move inside folder
shutil.move(copy_target, move_target)
print("Moved to", move_target)


### 🏃‍♂️ Exercise (medium): Simple backup directory

Write a script that:

1. Asks the user for a directory path `src_dir`.
2. Creates a backup directory called `src_dir + "_backup"` using `os.makedirs`.
3. Iterates over all files directly inside `src_dir` (ignore subdirectories).
4. Copies each file into the backup directory using `shutil.copy`.
5. Prints what was copied.

Use `os.listdir`, `os.path.isfile`, `os.path.join`, `os.makedirs`, and `shutil.copy`.


In [ ]:
# TODO: implement a simple backup directory copier.

#import os
#import shutil

# src_dir = input("Enter directory to backup: ")

# if not os.path.isdir(src_dir):
#     print("Not a directory.")
# else:
#     backup_dir = src_dir + "_backup"
#     os.makedirs(backup_dir, exist_ok=True)
#     for name in os.listdir(src_dir):
#         full_src = os.path.join(src_dir, name)
#         if os.path.isfile(full_src):
#             full_dst = os.path.join(backup_dir, name)
#             # copy file
#             ...
#             print(f"Copied {full_src!r} to {full_dst!r}")


In [ ]:
# Reference solution: simple backup directory

import os
import shutil

src_dir = "day4_shutil"  # replace with input(...) for interactive use

if not os.path.isdir(src_dir):
    print("Not a directory.")
else:
    backup_dir = src_dir + "_backup"
    os.makedirs(backup_dir, exist_ok=True)
    for name in os.listdir(src_dir):
        full_src = os.path.join(src_dir, name)
        if os.path.isfile(full_src):
            full_dst = os.path.join(backup_dir, name)
            shutil.copy(full_src, full_dst)
            print(f"Copied {full_src!r} to {full_dst!r}")


## 13. Temporary files with tempfile

Sometimes you need a file only for a short time (for example, to store intermediate results). The `tempfile` module helps create such temporary files and directories.

Common functions:

- `tempfile.TemporaryFile()` - creates a temporary file object
- `tempfile.NamedTemporaryFile()` - creates a temp file with a real name on disk
- `tempfile.TemporaryDirectory()` - creates a temporary directory

These objects are usually used as context managers and clean up automatically.

```python
import tempfile
with tempfile.NamedTemporaryFile(mode="w+", encoding="utf-8", delete=True) as tmp:
    tmp.write("hello\n")
    tmp.seek(0)
    print(tmp.read())
```

Documentation: https://docs.python.org/3/library/tempfile.html


In [ ]:
# Example: using NamedTemporaryFile

import tempfile

with tempfile.NamedTemporaryFile(mode="w+", encoding="utf-8", delete=True) as tmp:
    print("Temporary file name:", tmp.name)
    tmp.write("Temporary content\n")
    tmp.seek(0)
    data = tmp.read()
    print("Read back:")
    print(data)

print("After the with block, the temporary file is deleted.")


### 🧪 Exercise (medium): Write and read a temporary log file

Using `tempfile.NamedTemporaryFile`, write a script that:

1. Creates a named temporary file in text mode.
2. Writes a few log lines like `"INFO: Start"`, `"INFO: Working"`, `"INFO: Done"`.
3. Seeks back to the beginning.
4. Reads all lines and prints them.

Do all of this inside a `with` block so the file is cleaned up automatically.


In [ ]:
# TODO: write and read a temporary log file.

#import tempfile

# with tempfile.NamedTemporaryFile(mode="w+", encoding="utf-8", delete=True) as tmp:
#     # write some log lines
#     ...
#     # go back to start
#     ...
#     # read and print
#     ...

# print("Temporary log file should now be deleted.")


In [ ]:
# Reference solution: temporary log file

import tempfile

with tempfile.NamedTemporaryFile(mode="w+", encoding="utf-8", delete=True) as tmp:
    tmp.write("INFO: Start\n")
    tmp.write("INFO: Working\n")
    tmp.write("INFO: Done\n")
    tmp.seek(0)
    print("Log contents:")
    for line in tmp:
        print(line.strip())

print("Temporary log file should now be deleted.")


---
# Short break (14:45-15:00)

---

## 14. Introduction to HTTP and web APIs

HTTP (Hypertext Transfer Protocol) is the basic protocol of the web.

Key ideas:

- A **client** (your Python script, a browser) sends a **request** to a **server**.
- The request has:
  - A method: `GET`, `POST`, `PUT`, `DELETE`, ...
  - A URL: for example `https://api.example.com/data`
  - Optional headers and body (for example JSON data)
- The server sends back a **response** with:
  - A status code: `200 OK`, `404 Not Found`, `500 Internal Server Error`, ...
  - Headers
  - Body (HTML, JSON, binary, ...)

In Python, we often use the `requests` library to work with HTTP APIs in a convenient way.

Good HTTP overview: https://developer.mozilla.org/en-US/docs/Web/HTTP/Overview


## 15. Simple HTTP requests with the requests library

The `requests` library makes HTTP calls easy.

Basic GET request:

```python
import requests
response = requests.get("https://httpbin.org/get")
print(response.status_code)
print(response.text)
```

For JSON APIs:

```python
data = response.json()
```

Requests docs: https://requests.readthedocs.io/en/latest/


In [ ]:
# Example: simple GET request to httpbin

import requests

url = "https://httpbin.org/get"
response = requests.get(url)

print("Status:", response.status_code)

# Print part of the JSON response
if response.headers.get("Content-Type", "").startswith("application/json"):
    data = response.json()
    print("You sent these headers:")
    # Only print a few keys
    for key in list(data.get("headers", {}).keys())[:5]:
        print(f"  {key}: {data['headers'][key]}")
else:
    print("Response body:")
    print(response.text[:200])


### ✏ Exercise (easy): Fetch your origin IP from httpbin

Use `requests.get` to call `https://httpbin.org/ip` and:

1. Check that the status code is 200.
2. Parse the JSON body with `.json()`.
3. Print the value of the `origin` field.

Hint: the JSON looks like `{"origin": "..."}`.


In [ ]:
# TODO: fetch and print your origin IP from httpbin.

#import requests

# url = "https://httpbin.org/ip"
# response = requests.get(url)

# if response.status_code == 200:
#     data = ...  # parse JSON
#     origin = ...
#     print("Origin IP:", origin)
# else:
#     print("Request failed with status", response.status_code)


In [ ]:
# Reference solution: fetch origin IP from httpbin

import requests

url = "https://httpbin.org/ip"
response = requests.get(url)

if response.status_code == 200:
    data = response.json()
    origin = data.get("origin")
    print("Origin IP:", origin)
else:
    print("Request failed with status", response.status_code)


## 16. OpenRouter registration (LLM API)

OpenRouter is a service that allows you to access various large language models via a unified HTTP API.

Typical steps to use it from Python:

1. Go to https://openrouter.ai/ and create an account.
2. Generate an API key in your account settings.
3. Store that API key securely (for example, in an environment variable like `OPENROUTER_API_KEY`).
4. Use `requests.post` to call the `/api/v1/chat/completions` endpoint.

Never hard-code real API keys into code that will be shared publicly or committed to version control.


## 17. OpenRouter LLM call with requests

Here is a minimal example of calling an LLM on OpenRouter.
We assume you have set an environment variable `OPENROUTER_API_KEY` with your secret key.

```python
import os
import requests

api_key = os.environ.get("OPENROUTER_API_KEY")

headers = {
    "Authorization": f"Bearer {api_key}",
    "Content-Type": "application/json",
}

payload = {
    "model": "openai/gpt-4.1-mini",  # or another supported model
    "messages": [
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": "Say hello from a Python script."},
    ],
}

response = requests.post("https://openrouter.ai/api/v1/chat/completions", headers=headers, json=payload)
data = response.json()
print(data["choices"][0]["message"]["content"])
```

OpenRouter API docs: https://openrouter.ai/docs/api-reference/chat-completions


In [ ]:
# Example: helper function to call OpenRouter (will only work if you have an API key set)

import os
import requests


def call_openrouter(prompt):
    """Call OpenRouter with a simple prompt and return the text response.

    Requires the environment variable OPENROUTER_API_KEY to be set.
    """
    api_key = os.environ.get("OPENROUTER_API_KEY")
    if not api_key:
        print("OPENROUTER_API_KEY is not set. Skipping real API call.")
        return "(no response - missing API key)"

    headers = {
        "Authorization": f"Bearer {api_key}",
        "Content-Type": "application/json",
    }

    payload = {
        "model": "openai/gpt-4.1-mini",
        "messages": [
            {"role": "system", "content": "You are a concise assistant."},
            {"role": "user", "content": prompt},
        ],
    }

    response = requests.post("https://openrouter.ai/api/v1/chat/completions", headers=headers, json=payload, timeout=30)
    response.raise_for_status()
    data = response.json()
    return data["choices"][0]["message"]["content"]


if __name__ == "__main__":
    result = call_openrouter("Give me one short sentence about Python.")
    print("OpenRouter response:")
    print(result)


### 💪 Exercise (advanced): Ask the LLM to summarize a short text

Using the `call_openrouter(prompt)` helper (or a similar one you write), create a small script that:

1. Asks the user for a short paragraph of text.
2. Builds a prompt like: `"Summarize this in one sentence: <user text>"`.
3. Passes the prompt to `call_openrouter`.
4. Prints the returned summary.

If you do not have an API key, you can still implement the logic and then verify it later when you get access.


In [ ]:
# TODO: ask the user for text and summarize with OpenRouter.

# def summarize_text_with_llm(text):
#     prompt = "Summarize this in one sentence: " + text
#     # use call_openrouter(prompt)
#     ...

# user_text = "Python is a versatile language used for many tasks."  # replace with input(...)
# summary = summarize_text_with_llm(user_text)
# print("Summary:", summary)


In [ ]:
# Reference solution: ask user text and summarize with OpenRouter

# We assume call_openrouter is already defined in this notebook.


def summarize_text_with_llm(text):
    prompt = "Summarize this in one sentence: " + text
    return call_openrouter(prompt)


user_text = "Python is a versatile language used for web development, data science, automation, and more."  # replace with input(...)
summary = summarize_text_with_llm(user_text)
print("Summary:", summary)


## 18. Complex combined example: Daily notes archiver

In this final example we combine concepts from previous days:

- Input and output with `input()` and `print()` (Day 1)
- Lists, loops, comprehensions (Day 2)
- Functions and error handling (Day 3)
- File and directory operations, metadata, and paths (Day 4)

### Task

Create a small **Daily notes archiver** that:

1. Uses `pathlib.Path` to work in a base directory called `notes`.
2. Asks the user for a note category, for example `"work"` or `"personal"`.
3. Creates a subdirectory `notes/<category>` if it does not exist.
4. Asks the user for a note text and saves it into a new file with a name based on the current timestamp, for example `note_20251117_153045.txt`.
5. After saving, lists all note files in that category directory using `.glob("*.txt")`.
6. For each note file, prints the filename and its size in bytes (using `os.stat` or `Path.stat()`).
7. Optionally (bonus): if there are more than 5 note files, move the oldest ones into an `archive` subdirectory using `shutil.move`.

Use functions where it makes sense, and handle errors (for example, invalid category names or file issues) with try/except if you want to practice.


In [ ]:
# TODO: implement the Daily notes archiver.

#from pathlib import Path
#import os
#import shutil
#import time

# def get_timestamp_string():
#     # return a string like 20251117_153045
#     ...

# def ensure_category_dir(base, category):
#     # create notes/<category> if it does not exist, return the Path
#     ...

# def save_note(category_dir, text):
#     # create a new file with timestamped name and write the note
#     ...

# def list_notes_with_sizes(category_dir):
#     # list all .txt files and print their size
#     ...

# def maybe_archive_old_notes(category_dir, max_files=5):
#     # bonus: move oldest files to category_dir / "archive" if there are more than max_files
#     ...

# def main():
#     base = Path("notes")
#     base.mkdir(exist_ok=True)
#     category = input("Enter note category (e.g. work, personal): ").strip()
#     if not category:
#         print("Empty category, aborting.")
#         return
#     category_dir = ensure_category_dir(base, category)
#     text = input("Enter your note: ")
#     save_note(category_dir, text)
#     print("Notes in this category:")
#     list_notes_with_sizes(category_dir)
#     # bonus
#     # maybe_archive_old_notes(category_dir)

# if __name__ == "__main__":
#     main()


In [ ]:
# Reference solution: Daily notes archiver

from pathlib import Path
import os
import shutil
import time


def get_timestamp_string():
    """Return a timestamp string like 20251117_153045."""
    return time.strftime("%Y%m%d_%H%M%S")


def ensure_category_dir(base, category):
    """Create notes/<category> if needed and return its Path."""
    category_dir = base / category
    category_dir.mkdir(parents=True, exist_ok=True)
    return category_dir


def save_note(category_dir, text):
    """Save a note to a new timestamped file in category_dir."""
    timestamp = get_timestamp_string()
    filename = f"note_{timestamp}.txt"
    path = category_dir / filename
    with path.open("w", encoding="utf-8") as f:
        f.write(text + "\n")
    print("Saved note to", path)
    return path


def list_notes_with_sizes(category_dir):
    """Print all note files and their sizes in bytes."""
    files = sorted(category_dir.glob("*.txt"))
    if not files:
        print("No notes yet.")
        return files
    for path in files:
        size = path.stat().st_size
        print(f"{path.name}: {size} bytes")
    return files


def maybe_archive_old_notes(category_dir, max_files=5):
    """If there are more than max_files notes, move the oldest ones to archive/.

    Oldest is determined by modification time.
    """
    files = list(category_dir.glob("*.txt"))
    if len(files) <= max_files:
        return
    files.sort(key=lambda p: p.stat().st_mtime)  # oldest first
    archive_dir = category_dir / "archive"
    archive_dir.mkdir(exist_ok=True)
    to_archive = files[:-max_files]
    for path in to_archive:
        target = archive_dir / path.name
        shutil.move(str(path), str(target))
        print("Archived", path.name, "to", target)


def main():
    base = Path("notes")
    base.mkdir(exist_ok=True)

    category = "demo"  # replace with input("Enter note category (e.g. work, personal): ").strip()
    if not category:
        print("Empty category, aborting.")
        return

    category_dir = ensure_category_dir(base, category)

    text = "This is a demo note."  # replace with input("Enter your note: ")
    save_note(category_dir, text)

    print("Notes in this category:")
    files = list_notes_with_sizes(category_dir)

    # bonus: archive if many notes
    maybe_archive_old_notes(category_dir, max_files=5)


if __name__ == "__main__":
    main()


## Day 4 summary

Today you learned how to:

- Use common file methods: `open`, `read`, `readline`, `readlines`, `write`, `writelines`, `close`
- Use `with open(...)` context managers for safe file handling
- List directories and traverse directory trees with `os.listdir` and `os.walk`
- Create nested directories with `os.makedirs`
- Delete and rename files with `os.remove` and `os.rename`
- Check file and directory existence with `os.path.exists`, `os.path.isfile`, `os.path.isdir`
- Inspect file metadata (size, modification time) with `os.stat`
- Use `pathlib.Path` for object-oriented path handling, `.exists()`, `.is_file()`, `.is_dir()`, `.glob()`
- Copy and move files with `shutil.copy` and `shutil.move`
- Create and use temporary files with `tempfile.NamedTemporaryFile`
- Understand the basics of HTTP: requests, responses, methods, status codes
- Use the `requests` library for simple GET calls and parse JSON responses
- Understand how to register for OpenRouter and how to call an LLM API with `requests.post`
- Combine file, directory, and API concepts in a small Daily notes archiver project

These tools are fundamental for many practical Python tasks: processing data files, building automation scripts, and integrating with web services and LLMs.
